##### For this experiment I am going to drop "Classification" column. The goal is to test how it impacts the training. The thought process behind is that this data attribute may not be relevant to the applicant status. This feature is more like an identification because this is a goverment organization classification.
##### So with this reduced data set I am going to go through the same steps that we used in the first iteration.

In [2]:
# Import our dependencies

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import pandas as pd
import tensorflow as tf
from sklearn.linear_model import LogisticRegression

In [3]:
# Import our input dataset
charity_df = pd.read_csv('charity_data.csv')
charity_df

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,996010315,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,996012607,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,996015768,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [4]:
# Checking null values
for column in charity_df.columns:
    print(f"Column {column} has {charity_df[column].isnull().sum()} null values")

Column EIN has 0 null values
Column NAME has 0 null values
Column APPLICATION_TYPE has 0 null values
Column AFFILIATION has 0 null values
Column CLASSIFICATION has 0 null values
Column USE_CASE has 0 null values
Column ORGANIZATION has 0 null values
Column STATUS has 0 null values
Column INCOME_AMT has 0 null values
Column SPECIAL_CONSIDERATIONS has 0 null values
Column ASK_AMT has 0 null values
Column IS_SUCCESSFUL has 0 null values


In [5]:
charity_df.dtypes

EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [6]:
charity_df["INCOME_AMT"].unique()

array(['0', '1-9999', '10000-24999', '100000-499999', '10M-50M',
       '25000-99999', '50M+', '1M-5M', '5M-10M'], dtype=object)

In [7]:
charity_df["APPLICATION_TYPE"].unique()

array(['T10', 'T3', 'T5', 'T7', 'T4', 'T6', 'T2', 'T9', 'T19', 'T8',
       'T13', 'T12', 'T29', 'T25', 'T14', 'T17', 'T15'], dtype=object)

In [8]:
charity_df.nunique()

EIN                       34299
NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [9]:
charity_df["CLASSIFICATION"].unique()

array(['C1000', 'C2000', 'C3000', 'C1200', 'C2700', 'C7000', 'C7200',
       'C1700', 'C4000', 'C7100', 'C2800', 'C6000', 'C2100', 'C1238',
       'C5000', 'C7120', 'C1800', 'C4100', 'C1400', 'C1270', 'C2300',
       'C8200', 'C1500', 'C7210', 'C1300', 'C1230', 'C1280', 'C1240',
       'C2710', 'C2561', 'C1250', 'C8000', 'C1245', 'C1260', 'C1235',
       'C1720', 'C1257', 'C4500', 'C2400', 'C8210', 'C1600', 'C1278',
       'C1237', 'C4120', 'C2170', 'C1728', 'C1732', 'C2380', 'C1283',
       'C1570', 'C2500', 'C1267', 'C3700', 'C1580', 'C2570', 'C1256',
       'C1236', 'C1234', 'C1246', 'C2190', 'C4200', 'C0', 'C3200',
       'C5200', 'C1370', 'C2600', 'C1248', 'C6100', 'C1820', 'C1900',
       'C2150'], dtype=object)

Splitting the df to two - categorical and numerical, defininf the target and features, and dropping the columns that are neither 

In [10]:
# Dropping redundant data. 
# As I think the identification columns such as EIN and NAME 
# are not making any features in our dataset. AND I am dropping 
# the Classification as well to see if it will make a difference.
charity_reduced_df = charity_df.drop(columns=["EIN","NAME", "CLASSIFICATION"], axis=1)
charity_reduced_df

,APPLICATION_TYPE,AFFILIATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,ProductDev,Association,1,0,N,5000,1


In [12]:
# Creating a numeric df
numeric_columns = charity_reduced_df.dtypes[charity_reduced_df.dtypes == "int64"].index.tolist()
numeric_columns

['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL']

In [13]:
# Keeping only the 'STATUS' and 'ASK_AMT'. 
# The 'IS_SUCCESSFUL' will be the target
numeric_columns_df = charity_reduced_df[['STATUS', 'ASK_AMT']]
numeric_columns_df

,STATUS,ASK_AMT
0,1,5000
1,1,108590
2,1,5000
3,1,6692
4,1,142590
...,...,...
34294,1,5000
34295,1,5000
34296,1,5000
34297,1,5000


In [14]:
# Creating a categorical df - features that have to be encoded
categ_columns = charity_reduced_df.dtypes[charity_reduced_df.dtypes == "object"].index.tolist()
categ_columns

['APPLICATION_TYPE',
 'AFFILIATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [15]:
categ_columns_df = charity_reduced_df[['APPLICATION_TYPE',
 'AFFILIATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']]
categ_columns_df

,APPLICATION_TYPE,AFFILIATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS
0,T10,Independent,ProductDev,Association,0,N
1,T3,Independent,Preservation,Co-operative,1-9999,N
2,T5,CompanySponsored,ProductDev,Association,0,N
3,T3,CompanySponsored,Preservation,Trust,10000-24999,N
4,T3,Independent,Heathcare,Trust,100000-499999,N
...,...,...,...,...,...,...
34294,T4,Independent,ProductDev,Association,0,N
34295,T4,CompanySponsored,ProductDev,Association,0,N
34296,T3,CompanySponsored,Preservation,Association,0,N
34297,T5,Independent,ProductDev,Association,0,N


In [16]:
# Defining the target
y = charity_reduced_df['IS_SUCCESSFUL']
y

0        1
1        1
2        0
3        1
4        1
        ..
34294    0
34295    0
34296    0
34297    1
34298    0
Name: IS_SUCCESSFUL, Length: 34299, dtype: int64

In [17]:
# Encoding the categorical features

# Creating a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fitting and transforming the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(categ_columns_df))
encode_df

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34295,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34296,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34297,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [18]:
# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(categ_columns)
encode_df.head()

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [19]:
# Merging encoded categorical data with numerical df
X = numeric_columns_df.merge(encode_df, left_index=True, right_index=True)
X.head()

,STATUS,ASK_AMT,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


Now the features dataset has to be split to training and testing data, and then to be normalized

In [20]:
# Split X and y into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [21]:
# Scale X_train and X_test
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [22]:
X_train_scaled

array([[ 0.01394306, -0.03348439, -0.12647487, ..., -0.06987849,
         0.02789425, -0.02789425],
       [ 0.01394306, -0.03348439, -0.12647487, ..., -0.06987849,
         0.02789425, -0.02789425],
       [ 0.01394306, -0.03348439, -0.12647487, ..., -0.06987849,
         0.02789425, -0.02789425],
       ...,
       [ 0.01394306, -0.03229417, -0.12647487, ..., -0.06987849,
         0.02789425, -0.02789425],
       [ 0.01394306, -0.03348439, -0.12647487, ..., -0.06987849,
         0.02789425, -0.02789425],
       [ 0.01394306, -0.01093629, -0.12647487, ..., -0.06987849,
         0.02789425, -0.02789425]])

In [23]:
X_test_scaled

array([[ 1.39430557e-02, -3.33676580e-02, -1.26474869e-01, ...,
        -6.98784891e-02,  2.78942470e-02, -2.78942470e-02],
       [ 1.39430557e-02,  2.86802949e-02, -1.26474869e-01, ...,
         1.43105555e+01,  2.78942470e-02, -2.78942470e-02],
       [ 1.39430557e-02, -3.34843874e-02, -1.26474869e-01, ...,
        -6.98784891e-02,  2.78942470e-02, -2.78942470e-02],
       ...,
       [ 1.39430557e-02, -3.34843874e-02, -1.26474869e-01, ...,
        -6.98784891e-02,  2.78942470e-02, -2.78942470e-02],
       [ 1.39430557e-02, -3.34843874e-02, -1.26474869e-01, ...,
        -6.98784891e-02,  2.78942470e-02, -2.78942470e-02],
       [ 1.39430557e-02, -3.34843874e-02, -1.26474869e-01, ...,
        -6.98784891e-02,  2.78942470e-02, -2.78942470e-02]])

#### Selecting the approach

##### Will start with the Tanh activation function

First, will use one layer with 10 neurons

In [24]:
# Defining the model - one layer neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  10


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [25]:
# Train the model with 50 epochs
fit_model = nn.fit(X_train_scaled, y_train, epochs=50) 
# Evaluate the model using the test data 
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
25724/25724 [==============================] - 1s 36us/sample - loss: 0.6120 - acc: 0.6841
Epoch 2/50
25724/25724 [==============================] - 1s 32us/sample - loss: 0.5770 - acc: 0.7233
Epoch 3/50
25724/25724 [==============================] - 1s 31us/sample - loss: 0.5731 - acc: 0.7250
Epoch 4/50
25724/25724 [==============================] - 1s 31us/sample - loss: 0.5706 - acc: 0.7264
Epoch 5/50
25724/25724 [==============================] - 1s 32us/sample - loss: 0.5695 - acc: 0.7267
Epoch 6/50
25724/25724 [==============================] - 1s 35us/sample - loss: 0.5685 - acc: 0.7271
Epoch 7/50
25724/25724 [==============================] - 1s 33us/sample - loss: 0.5678 - acc: 0.7270
Epoch 8/50
25724/25724 [==============================] - 1s 34us/sample - loss: 0.5675 - acc: 0.7273
Epoch 9/50
25724/25724 [==============================] - 1s 32us/sample - loss: 0.5673 - acc: 0.7276
Epoch 10/50
25724/25724 [==============================] - 1s 31us/sample - loss: 

The loss is very high and the accuracy level is not impressive. So I would like to tyr increasing the number of neurones upto twice as much inputs we have.

In [26]:
len(X_train_scaled[0])

45

In [27]:
# Defining the model - one layer neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 90

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [28]:
# Train the model with 50 epochs
fit_model = nn.fit(X_train_scaled, y_train, epochs=50) 
# Evaluate the model using the test data 
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
25724/25724 [==============================] - 1s 35us/sample - loss: 0.5877 - acc: 0.7162
Epoch 2/50
25724/25724 [==============================] - 1s 32us/sample - loss: 0.5749 - acc: 0.7244
Epoch 3/50
25724/25724 [==============================] - 1s 33us/sample - loss: 0.5713 - acc: 0.7266
Epoch 4/50
25724/25724 [==============================] - 1s 34us/sample - loss: 0.5691 - acc: 0.7273
Epoch 5/50
25724/25724 [==============================] - 1s 33us/sample - loss: 0.5675 - acc: 0.7281
Epoch 6/50
25724/25724 [==============================] - 1s 33us/sample - loss: 0.5668 - acc: 0.7283
Epoch 7/50
25724/25724 [==============================] - 1s 33us/sample - loss: 0.5667 - acc: 0.7278
Epoch 8/50
25724/25724 [==============================] - 1s 33us/sample - loss: 0.5665 - acc: 0.7288
Epoch 9/50
25724/25724 [==============================] - 1s 33us/sample - loss: 0.5660 - acc: 0.7285
Epoch 10/50
25724/25724 [==============================] - 1s 34us/sample - loss: 

There is no improvement at all. I will increase the number of epochs but will apply it to the first case - with less neurons.

In [29]:
# Defining the model - one layer neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [30]:
# Train the model with 100 epochs
fit_model = nn.fit(X_train_scaled, y_train, epochs=100) 
# Evaluate the model using the test data 
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
25724/25724 [==============================] - 1s 36us/sample - loss: 0.5946 - acc: 0.7133
Epoch 2/100
25724/25724 [==============================] - 1s 34us/sample - loss: 0.5748 - acc: 0.7247
Epoch 3/100
25724/25724 [==============================] - 1s 32us/sample - loss: 0.5714 - acc: 0.7255
Epoch 4/100
25724/25724 [==============================] - 1s 32us/sample - loss: 0.5699 - acc: 0.7267
Epoch 5/100
25724/25724 [==============================] - 1s 32us/sample - loss: 0.5692 - acc: 0.7262
Epoch 6/100
25724/25724 [==============================] - 1s 32us/sample - loss: 0.5681 - acc: 0.7275
Epoch 7/100
25724/25724 [==============================] - 1s 32us/sample - loss: 0.5679 - acc: 0.7270
Epoch 8/100
25724/25724 [==============================] - 1s 33us/sample - loss: 0.5672 - acc: 0.7279
Epoch 9/100
25724/25724 [==============================] - 1s 31us/sample - loss: 0.5671 - acc: 0.7271
Epoch 10/100
25724/25724 [==============================] - 1s 33us/sampl

25724/25724 [==============================] - 1s 35us/sample - loss: 0.5627 - acc: 0.7304
Epoch 80/100
25724/25724 [==============================] - 1s 35us/sample - loss: 0.5626 - acc: 0.7300
Epoch 81/100
25724/25724 [==============================] - 1s 37us/sample - loss: 0.5625 - acc: 0.7301
Epoch 82/100
25724/25724 [==============================] - 1s 35us/sample - loss: 0.5625 - acc: 0.7295
Epoch 83/100
25724/25724 [==============================] - 1s 34us/sample - loss: 0.5624 - acc: 0.7296
Epoch 84/100
25724/25724 [==============================] - 1s 34us/sample - loss: 0.5625 - acc: 0.7292
Epoch 85/100
25724/25724 [==============================] - 1s 34us/sample - loss: 0.5625 - acc: 0.7304
Epoch 86/100
25724/25724 [==============================] - 1s 33us/sample - loss: 0.5626 - acc: 0.7302
Epoch 87/100
25724/25724 [==============================] - 1s 33us/sample - loss: 0.5624 - acc: 0.7306
Epoch 88/100
25724/25724 [==============================] - 1s 33us/sample - 

Will use the combination of 100 epochs and number of neurons equal to double size of inputs. If it doesn't work will move to an additional layer.

In [31]:
# Defining the model - one layer neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 232

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [32]:
# Train the model with 100 epochs
fit_model = nn.fit(X_train_scaled, y_train, epochs=100) 
# Evaluate the model using the test data 
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
25724/25724 [==============================] - 1s 41us/sample - loss: 0.5894 - acc: 0.7177
Epoch 2/100
25724/25724 [==============================] - 1s 37us/sample - loss: 0.5793 - acc: 0.7243
Epoch 3/100
25724/25724 [==============================] - 1s 39us/sample - loss: 0.5746 - acc: 0.7252
Epoch 4/100
25724/25724 [==============================] - 1s 39us/sample - loss: 0.5710 - acc: 0.7278
Epoch 5/100
25724/25724 [==============================] - 1s 37us/sample - loss: 0.5694 - acc: 0.7274
Epoch 6/100
25724/25724 [==============================] - 1s 38us/sample - loss: 0.5688 - acc: 0.7261
Epoch 7/100
25724/25724 [==============================] - 1s 38us/sample - loss: 0.5682 - acc: 0.7281
Epoch 8/100
25724/25724 [==============================] - 1s 38us/sample - loss: 0.5672 - acc: 0.7258
Epoch 9/100
25724/25724 [==============================] - 1s 38us/sample - loss: 0.5672 - acc: 0.7279
Epoch 10/100
25724/25724 [==============================] - 1s 37us/sampl

25724/25724 [==============================] - 1s 41us/sample - loss: 0.5612 - acc: 0.7320
Epoch 80/100
25724/25724 [==============================] - 1s 41us/sample - loss: 0.5617 - acc: 0.7299
Epoch 81/100
25724/25724 [==============================] - 1s 41us/sample - loss: 0.5616 - acc: 0.7300
Epoch 82/100
25724/25724 [==============================] - 1s 41us/sample - loss: 0.5612 - acc: 0.7301
Epoch 83/100
25724/25724 [==============================] - 1s 41us/sample - loss: 0.5614 - acc: 0.7297
Epoch 84/100
25724/25724 [==============================] - 1s 41us/sample - loss: 0.5616 - acc: 0.7301
Epoch 85/100
25724/25724 [==============================] - 1s 43us/sample - loss: 0.5613 - acc: 0.7298
Epoch 86/100
25724/25724 [==============================] - 1s 40us/sample - loss: 0.5613 - acc: 0.7304
Epoch 87/100
25724/25724 [==============================] - 1s 41us/sample - loss: 0.5611 - acc: 0.7309
Epoch 88/100
25724/25724 [==============================] - 1s 40us/sample - 

So it's not better again. Time to try one more layer. If it doesn't improve the result then I will rty the RELU activation function.

In [33]:
# Defining the model - one layer neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 10
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [34]:
# Train the model with 100 epochs
fit_model = nn.fit(X_train_scaled, y_train, epochs=100) 
# Evaluate the model using the test data 
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
25724/25724 [==============================] - 1s 40us/sample - loss: 0.5856 - acc: 0.7184
Epoch 2/100
25724/25724 [==============================] - 1s 36us/sample - loss: 0.5741 - acc: 0.7252
Epoch 3/100
25724/25724 [==============================] - 1s 36us/sample - loss: 0.5718 - acc: 0.7254
Epoch 4/100
25724/25724 [==============================] - 1s 38us/sample - loss: 0.5706 - acc: 0.7271
Epoch 5/100
25724/25724 [==============================] - 1s 38us/sample - loss: 0.5698 - acc: 0.7268
Epoch 6/100
25724/25724 [==============================] - 1s 37us/sample - loss: 0.5690 - acc: 0.7276
Epoch 7/100
25724/25724 [==============================] - 1s 36us/sample - loss: 0.5685 - acc: 0.7269
Epoch 8/100
25724/25724 [==============================] - 1s 36us/sample - loss: 0.5681 - acc: 0.7292
Epoch 9/100
25724/25724 [==============================] - 1s 36us/sample - loss: 0.5679 - acc: 0.7278
Epoch 10/100
25724/25724 [==============================] - 1s 36us/sampl

25724/25724 [==============================] - 1s 40us/sample - loss: 0.5620 - acc: 0.7307
Epoch 80/100
25724/25724 [==============================] - 1s 39us/sample - loss: 0.5620 - acc: 0.7295
Epoch 81/100
25724/25724 [==============================] - 1s 39us/sample - loss: 0.5619 - acc: 0.7308
Epoch 82/100
25724/25724 [==============================] - 1s 39us/sample - loss: 0.5620 - acc: 0.7301
Epoch 83/100
25724/25724 [==============================] - 1s 39us/sample - loss: 0.5621 - acc: 0.7303
Epoch 84/100
25724/25724 [==============================] - 1s 41us/sample - loss: 0.5619 - acc: 0.7309
Epoch 85/100
25724/25724 [==============================] - 1s 40us/sample - loss: 0.5618 - acc: 0.7312
Epoch 86/100
25724/25724 [==============================] - 1s 39us/sample - loss: 0.5620 - acc: 0.7301
Epoch 87/100
25724/25724 [==============================] - 1s 41us/sample - loss: 0.5618 - acc: 0.7302
Epoch 88/100
25724/25724 [==============================] - 1s 41us/sample - 

The result is not satisfying again.

##### RELU activation function.

Will start with one layer and 100 epochs.

In [35]:
# Defining the model - one layer neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 90

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [36]:
# Train the model with 100 epochs
fit_model = nn.fit(X_train_scaled, y_train, epochs=100) 
# Evaluate the model using the test data 
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
25724/25724 [==============================] - 1s 38us/sample - loss: 0.5859 - acc: 0.7189
Epoch 2/100
25724/25724 [==============================] - 1s 38us/sample - loss: 0.5734 - acc: 0.7257
Epoch 3/100
25724/25724 [==============================] - 1s 34us/sample - loss: 0.5707 - acc: 0.7260
Epoch 4/100
25724/25724 [==============================] - 1s 37us/sample - loss: 0.5697 - acc: 0.7268
Epoch 5/100
25724/25724 [==============================] - 1s 36us/sample - loss: 0.5690 - acc: 0.7257
Epoch 6/100
25724/25724 [==============================] - 1s 35us/sample - loss: 0.5682 - acc: 0.7275
Epoch 7/100
25724/25724 [==============================] - 1s 35us/sample - loss: 0.5677 - acc: 0.7275
Epoch 8/100
25724/25724 [==============================] - 1s 36us/sample - loss: 0.5673 - acc: 0.7273
Epoch 9/100
25724/25724 [==============================] - 1s 36us/sample - loss: 0.5672 - acc: 0.7275
Epoch 10/100
25724/25724 [==============================] - 1s 35us/sampl

25724/25724 [==============================] - 1s 41us/sample - loss: 0.5621 - acc: 0.7303
Epoch 80/100
25724/25724 [==============================] - 1s 42us/sample - loss: 0.5621 - acc: 0.7304
Epoch 81/100
25724/25724 [==============================] - 1s 41us/sample - loss: 0.5621 - acc: 0.7306
Epoch 82/100
25724/25724 [==============================] - 1s 41us/sample - loss: 0.5619 - acc: 0.7303
Epoch 83/100
25724/25724 [==============================] - 1s 41us/sample - loss: 0.5619 - acc: 0.7300
Epoch 84/100
25724/25724 [==============================] - 1s 42us/sample - loss: 0.5623 - acc: 0.7311
Epoch 85/100
25724/25724 [==============================] - 1s 41us/sample - loss: 0.5619 - acc: 0.7309
Epoch 86/100
25724/25724 [==============================] - 1s 41us/sample - loss: 0.5615 - acc: 0.7302
Epoch 87/100
25724/25724 [==============================] - 1s 42us/sample - loss: 0.5621 - acc: 0.7303
Epoch 88/100
25724/25724 [==============================] - 1s 42us/sample - 

Adding one more layer but with less neurons.

In [37]:
# Defining the model - one layer neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 10
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [38]:
# Train the model with 100 epochs
fit_model = nn.fit(X_train_scaled, y_train, epochs=100) 
# Evaluate the model using the test data 
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
25724/25724 [==============================] - 1s 42us/sample - loss: 0.6137 - acc: 0.6986
Epoch 2/100
25724/25724 [==============================] - 1s 37us/sample - loss: 0.5753 - acc: 0.7236
Epoch 3/100
25724/25724 [==============================] - 1s 37us/sample - loss: 0.5717 - acc: 0.7253
Epoch 4/100
25724/25724 [==============================] - 1s 37us/sample - loss: 0.5700 - acc: 0.7245
Epoch 5/100
25724/25724 [==============================] - 1s 37us/sample - loss: 0.5689 - acc: 0.7264
Epoch 6/100
25724/25724 [==============================] - 1s 38us/sample - loss: 0.5682 - acc: 0.7264
Epoch 7/100
25724/25724 [==============================] - 1s 39us/sample - loss: 0.5675 - acc: 0.7263
Epoch 8/100
25724/25724 [==============================] - 1s 41us/sample - loss: 0.5672 - acc: 0.7267
Epoch 9/100
25724/25724 [==============================] - 1s 39us/sample - loss: 0.5669 - acc: 0.7262
Epoch 10/100
25724/25724 [==============================] - 1s 45us/sampl

25724/25724 [==============================] - 1s 44us/sample - loss: 0.5626 - acc: 0.7296
Epoch 80/100
25724/25724 [==============================] - 1s 43us/sample - loss: 0.5629 - acc: 0.7298
Epoch 81/100
25724/25724 [==============================] - 1s 43us/sample - loss: 0.5627 - acc: 0.7292
Epoch 82/100
25724/25724 [==============================] - 1s 42us/sample - loss: 0.5628 - acc: 0.7296
Epoch 83/100
25724/25724 [==============================] - 1s 43us/sample - loss: 0.5630 - acc: 0.7296
Epoch 84/100
25724/25724 [==============================] - 1s 42us/sample - loss: 0.5627 - acc: 0.7305
Epoch 85/100
25724/25724 [==============================] - 1s 43us/sample - loss: 0.5631 - acc: 0.7288
Epoch 86/100
25724/25724 [==============================] - 1s 45us/sample - loss: 0.5626 - acc: 0.7298
Epoch 87/100
25724/25724 [==============================] - 1s 45us/sample - loss: 0.5628 - acc: 0.7295
Epoch 88/100
25724/25724 [==============================] - 1s 46us/sample - 

Increasing the number of neurons.

In [39]:
# Defining the model - one layer neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 90
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [40]:
# Train the model with 100 epochs
fit_model = nn.fit(X_train_scaled, y_train, epochs=100) 
# Evaluate the model using the test data 
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
25724/25724 [==============================] - 1s 46us/sample - loss: 0.5798 - acc: 0.7220
Epoch 2/100
25724/25724 [==============================] - 1s 41us/sample - loss: 0.5723 - acc: 0.7263
Epoch 3/100
25724/25724 [==============================] - 1s 42us/sample - loss: 0.5697 - acc: 0.7271
Epoch 4/100
25724/25724 [==============================] - 1s 41us/sample - loss: 0.5682 - acc: 0.7266
Epoch 5/100
25724/25724 [==============================] - 1s 41us/sample - loss: 0.5680 - acc: 0.7269
Epoch 6/100
25724/25724 [==============================] - 1s 41us/sample - loss: 0.5665 - acc: 0.7281
Epoch 7/100
25724/25724 [==============================] - 1s 43us/sample - loss: 0.5665 - acc: 0.7269
Epoch 8/100
25724/25724 [==============================] - 1s 41us/sample - loss: 0.5660 - acc: 0.7283
Epoch 9/100
25724/25724 [==============================] - 1s 41us/sample - loss: 0.5653 - acc: 0.7296
Epoch 10/100
25724/25724 [==============================] - 1s 42us/sampl

25724/25724 [==============================] - 1s 49us/sample - loss: 0.5597 - acc: 0.7321
Epoch 80/100
25724/25724 [==============================] - 1s 46us/sample - loss: 0.5595 - acc: 0.7311
Epoch 81/100
25724/25724 [==============================] - 1s 47us/sample - loss: 0.5599 - acc: 0.7319
Epoch 82/100
25724/25724 [==============================] - 1s 47us/sample - loss: 0.5603 - acc: 0.7311
Epoch 83/100
25724/25724 [==============================] - 1s 46us/sample - loss: 0.5598 - acc: 0.7320
Epoch 84/100
25724/25724 [==============================] - 1s 48us/sample - loss: 0.5597 - acc: 0.7321
Epoch 85/100
25724/25724 [==============================] - 1s 46us/sample - loss: 0.5597 - acc: 0.7313
Epoch 86/100
25724/25724 [==============================] - 1s 49us/sample - loss: 0.5597 - acc: 0.7318
Epoch 87/100
25724/25724 [==============================] - 1s 48us/sample - loss: 0.5595 - acc: 0.7319
Epoch 88/100
25724/25724 [==============================] - 1s 52us/sample - 

#### Trying other options - linear activation function in the output layer.

In [54]:
# Defining the model - one layer neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 232
hidden_nodes_layer2 = 100


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="linear"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])

In [55]:
# Train the model with 100 epochs
fit_model = nn.fit(X_train_scaled, y_train, epochs=100) 
# Evaluate the model using the test data 
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
25724/25724 [==============================] - 1s 56us/sample - loss: 0.2794 - acc: 0.7104
Epoch 2/100
25724/25724 [==============================] - 1s 53us/sample - loss: 0.2441 - acc: 0.7241
Epoch 3/100
25724/25724 [==============================] - 1s 53us/sample - loss: 0.2005 - acc: 0.7252
Epoch 4/100
25724/25724 [==============================] - 1s 53us/sample - loss: 0.1928 - acc: 0.7266
Epoch 5/100
25724/25724 [==============================] - 1s 53us/sample - loss: 0.1929 - acc: 0.7261
Epoch 6/100
25724/25724 [==============================] - 1s 52us/sample - loss: 0.1926 - acc: 0.7269
Epoch 7/100
25724/25724 [==============================] - 1s 53us/sample - loss: 0.1921 - acc: 0.72851s - 
Epoch 8/100
25724/25724 [==============================] - 1s 55us/sample - loss: 0.1924 - acc: 0.7255
Epoch 9/100
25724/25724 [==============================] - 1s 52us/sample - loss: 0.1923 - acc: 0.7289
Epoch 10/100
25724/25724 [==============================] - 1s 52us/

25724/25724 [==============================] - 1s 53us/sample - loss: 0.1890 - acc: 0.7314
Epoch 80/100
25724/25724 [==============================] - 1s 54us/sample - loss: 0.1890 - acc: 0.7317
Epoch 81/100
25724/25724 [==============================] - 1s 55us/sample - loss: 0.1889 - acc: 0.7306
Epoch 82/100
25724/25724 [==============================] - 1s 53us/sample - loss: 0.1895 - acc: 0.7308
Epoch 83/100
25724/25724 [==============================] - 1s 53us/sample - loss: 0.1891 - acc: 0.7301
Epoch 84/100
25724/25724 [==============================] - 1s 52us/sample - loss: 0.1890 - acc: 0.7306
Epoch 85/100
25724/25724 [==============================] - 1s 53us/sample - loss: 0.1894 - acc: 0.7297
Epoch 86/100
25724/25724 [==============================] - 1s 52us/sample - loss: 0.1892 - acc: 0.7303
Epoch 87/100
25724/25724 [==============================] - 1s 53us/sample - loss: 0.1891 - acc: 0.7315
Epoch 88/100
25724/25724 [==============================] - 1s 53us/sample - 

In [57]:
# Defining the model - one layer neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 232
hidden_nodes_layer2 = 100
hidden_nodes_layer3 = 50

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="linear"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])

In [58]:
# Train the model with 100 epochs
fit_model = nn.fit(X_train_scaled, y_train, epochs=100) 
# Evaluate the model using the test data 
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
25724/25724 [==============================] - 2s 61us/sample - loss: 0.2289 - acc: 0.7142
Epoch 2/100
25724/25724 [==============================] - 2s 61us/sample - loss: 0.2152 - acc: 0.7223
Epoch 3/100
25724/25724 [==============================] - 2s 61us/sample - loss: 0.2047 - acc: 0.7241
Epoch 4/100
25724/25724 [==============================] - 1s 56us/sample - loss: 0.1975 - acc: 0.7240
Epoch 5/100
25724/25724 [==============================] - 2s 59us/sample - loss: 0.1929 - acc: 0.7259
Epoch 6/100
25724/25724 [==============================] - 1s 56us/sample - loss: 0.1926 - acc: 0.7265
Epoch 7/100
25724/25724 [==============================] - 1s 56us/sample - loss: 0.1924 - acc: 0.72641s - 
Epoch 8/100
25724/25724 [==============================] - 1s 57us/sample - loss: 0.1922 - acc: 0.7270
Epoch 9/100
25724/25724 [==============================] - 1s 56us/sample - loss: 0.1917 - acc: 0.7269
Epoch 10/100
25724/25724 [==============================] - 1s 55us/

25724/25724 [==============================] - 2s 60us/sample - loss: 0.1892 - acc: 0.7306
Epoch 80/100
25724/25724 [==============================] - 2s 61us/sample - loss: 0.1889 - acc: 0.7312
Epoch 81/100
25724/25724 [==============================] - 1s 58us/sample - loss: 0.1902 - acc: 0.7314
Epoch 82/100
25724/25724 [==============================] - 1s 57us/sample - loss: 0.1898 - acc: 0.7309
Epoch 83/100
25724/25724 [==============================] - 1s 57us/sample - loss: 0.1892 - acc: 0.7296
Epoch 84/100
25724/25724 [==============================] - 1s 58us/sample - loss: 0.1892 - acc: 0.7307
Epoch 85/100
25724/25724 [==============================] - 1s 58us/sample - loss: 0.1890 - acc: 0.7309
Epoch 86/100
25724/25724 [==============================] - 2s 59us/sample - loss: 0.1890 - acc: 0.7310
Epoch 87/100
25724/25724 [==============================] - 1s 57us/sample - loss: 0.1889 - acc: 0.7309
Epoch 88/100
25724/25724 [==============================] - 1s 57us/sample - 

##### This data set without the "Classification" feature does not make the result better. So as I described in the previous analysis, this data set may not be perfect for funding success prediction. The accuracy level barely reaches 75%, it is max 74.5% in the data set with Classification feature included.

#### Let's see how Random forest and Logistic Regression work on this data set

Random forest.

In [41]:
from collections import Counter
Counter(y_train)

Counter({0: 12035, 1: 13689})

In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [44]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78) 

In [45]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [46]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [47]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,2638,1365
Actual 1,1185,3387


In [48]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)
acc_score

0.7026239067055393

In [49]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,2638,1365
Actual 1,1185,3387


Accuracy Score : 0.7026239067055393
Classification Report
              precision    recall  f1-score   support

           0       0.69      0.66      0.67      4003
           1       0.71      0.74      0.73      4572

    accuracy                           0.70      8575
   macro avg       0.70      0.70      0.70      8575
weighted avg       0.70      0.70      0.70      8575



Logistic Regression.

In [50]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', random_state=78)
classifier

LogisticRegression(random_state=78)

In [51]:
classifier.fit(X_train_scaled, y_train)
y_pred = classifier.predict(X_test_scaled)

In [52]:
matrix = confusion_matrix(y_test, y_pred)
matrix

array([[2662, 1341],
       [1070, 3502]], dtype=int64)

In [53]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.7154840016094666

##### Neither Random Forest or Logistic Regression does better job. So even though the neural network and deep learning result is not reaching 75%, it is still better than Supervised ML models.